# TensorFlow Network finding

This notebook will work finding the best trade-off for TensorFlow neural network, using previous data transformation

**Main goals are**: 
- Use our understanding blabla



In [3]:
IMAGE_SIZE = [163, 220]

In [4]:
from local_lib.dataset import retreiveDatasetFromCsv
from local_lib.flatten_transformer import FlattenTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import train_test_split
from local_lib.non_zero_transformer import NonZeroTransformer
from local_lib.mask_transformer import MaskSeuilTransformer
from local_lib.flatten_transformer import FlattenTransformer
from local_lib.dimension_transformer import DimensionTransformer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


dataset = retreiveDatasetFromCsv()
dataset = dataset.sample(frac=1, random_state=42)

In [5]:
pipeline = Pipeline([
    ('non_zero_tr', NonZeroTransformer()),
    ('mask_tr', MaskSeuilTransformer(strategy="combined")),
    ('minmax', MinMaxScaler())
    ])

In [6]:
# Fit and transform data
X = pipeline.fit_transform(dataset["pixel_value"])
y = np.array(list(dataset["class"]))


In [7]:
import copy

# Make a copy of the list
y_simpler = copy.deepcopy(y)
y_simpler[y_simpler == 2] = 1
np.unique(y_simpler)

y = y_simpler

In [8]:
print(X.shape)
print(y.shape)

(5856, 35860)
(5856,)


In [9]:
import keras

checkpoint_cb = keras.callbacks.ModelCheckpoint("xray_model.keras", save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)


In [10]:
initial_learning_rate = 0.015
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [11]:
#Reshape to 2D
X_reshaped = X.reshape((X.shape[0], IMAGE_SIZE[0], IMAGE_SIZE[1], 1))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)


In [14]:
X_reshaped.shape

(5856, 163, 220, 1)

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

c:\Users\a910572\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
model = Sequential([
    layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
    layers.Conv2D(16, 3, activation="relu", padding="same"),
    layers.Conv2D(16, 3, activation="relu", padding="same"),
    layers.MaxPool2D(),
    
    layers.SeparableConv2D(32, 3, activation="relu", padding="same"),
    layers.SeparableConv2D(32, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    
    layers.SeparableConv2D(64, 3, activation="relu", padding="same"),
    layers.SeparableConv2D(64, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    
    layers.SeparableConv2D(128, 3, activation="relu", padding="same"),
    layers.SeparableConv2D(128, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.2),
    
    layers.SeparableConv2D(256, 3, activation="relu", padding="same"),
    layers.SeparableConv2D(256, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPool2D(),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    
    layers.Dense(512, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.7),
    
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(64, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    
    layers.Dense(1, activation="sigmoid")
])

In [20]:
# Compile the model with the learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), 
              loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
# Train the model with the callbacks
history = model.fit(X_train, y_train, epochs=10, batch_size=32, 
                    validation_split=0.2, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 43s 362ms/step - accuracy: 0.9235 - loss: 0.2011 - val_accuracy: 0.3052 - val_loss: 24.8135
Epoch 2/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 46s 389ms/step - accuracy: 0.9160 - loss: 0.2281 - val_accuracy: 0.6201 - val_loss: 1.5039
Epoch 3/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 44s 374ms/step - accuracy: 0.9300 - loss: 0.2081 - val_accuracy: 0.7471 - val_loss: 1.4426
Epoch 4/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 383ms/step - accuracy: 0.9377 - loss: 0.1877 - val_accuracy: 0.9296 - val_loss: 0.1719
Epoch 5/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 46s 391ms/step - accuracy: 0.9265 - loss: 0.1953 - val_accuracy: 0.8975 - val_loss: 0.2560
Epoch 6/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 48s 409ms/step - accuracy: 0.9440 - loss: 0.1618 - val_accuracy: 0.9392 - val_loss: 0.1685
Epoch 7/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 48s 406ms/step - accuracy: 0.9240 - loss: 0.1838 - val_accuracy: 0.8463 - val_loss: 0.3943
Epoch 8/10
118/118 ━━━━━━━━━━━━━━━━━━━━ 45s 385ms/step - accuracy: 0.9374 - loss: 

In [23]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9486 - loss: 0.1436


In [24]:
print(f"Test accuracy: {test_acc}")

Test accuracy: 0.9453924894332886
